In [2]:
import asyncio
import json
import os
from typing import List, Dict, Any
from enum import Enum

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from graphiti_core import Graphiti


In [3]:
# Simple context quality assessment
class ContextQuality(Enum):
    INSUFFICIENT = "insufficient"
    SUFFICIENT = "sufficient"
    EXCESSIVE = "excessive"

In [4]:
# Simple context quality assessment
class ContextQuality(Enum):
    INSUFFICIENT = "insufficient"
    SUFFICIENT = "sufficient"
    EXCESSIVE = "excessive"

class SimpleContextAgent:
    def __init__(self):
        # Setup
        self.llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
        self.client = Graphiti(
            os.environ.get('NEO4J_URI', 'bolt://localhost:7687'),
            os.environ.get('NEO4J_USER', 'neo4j'),
            os.environ.get('NEO4J_PASSWORD', 'password')
        )
        
    async def get_context(self, query: str, max_results: int = 5) -> List[str]:
        """Get context from knowledge base"""
        try:
            results = await self.client.search(query, num_results=max_results)
            return [edge.fact for edge in results]
        except:
            return []
    
    async def assess_context_quality(self, query: str, context: List[str]) -> Dict[str, Any]:
        """Check if context is sufficient, insufficient, or excessive"""
        
        assessment_prompt = f"""
        Query: {query}
        
        Context Facts:
        {chr(10).join([f"- {fact}" for fact in context]) if context else "No context provided"}
        
        Assess the context quality for answering this query. Respond in JSON:
        {{
            "quality": "insufficient|sufficient|excessive",
            "confidence": 0.0-1.0,
            "reasoning": "brief explanation",
            "can_answer": true/false
        }}
        
        Guidelines:
        - insufficient: Missing key information needed to answer
        - sufficient: Has enough relevant information to answer well
        - excessive: Too much irrelevant information that clutters the response
        """
        
        try:
            response = await self.llm.ainvoke([HumanMessage(content=assessment_prompt)])
            return json.loads(response.content)
        except:
            # Fallback simple assessment
            if not context:
                return {"quality": "insufficient", "confidence": 1.0, "reasoning": "No context", "can_answer": False}
            elif len(context) > 10:
                return {"quality": "excessive", "confidence": 0.7, "reasoning": "Too many facts", "can_answer": True}
            else:
                return {"quality": "sufficient", "confidence": 0.8, "reasoning": "Reasonable amount", "can_answer": True}
    
    async def optimize_context(self, query: str, initial_context: List[str]) -> List[str]:
        """Get the right amount of context - not too much, not too little"""
        
        if not initial_context:
            # Try broader search
            broader_context = await self.get_context(query, max_results=10)
            return broader_context[:5]  # Cap at 5 for simplicity
        
        # Filter most relevant context
        filter_prompt = f"""
        Query: {query}
        
        Available Context:
        {chr(10).join([f"{i+1}. {fact}" for i, fact in enumerate(initial_context)])}
        
        Select the 3-5 most relevant facts that directly help answer the query.
        Return only the numbers (e.g., "1,3,5"):
        """
        
        try:
            response = await self.llm.ainvoke([HumanMessage(content=filter_prompt)])
            selected_nums = [int(n.strip()) for n in response.content.split(',') if n.strip().isdigit()]
            return [initial_context[i-1] for i in selected_nums if 0 < i <= len(initial_context)]
        except:
            return initial_context[:5]  # Fallback to first 5
    
    async def answer_with_context(self, query: str) -> Dict[str, Any]:
        """Main method: Get context, assess quality, and answer"""
        
        # Step 1: Get initial context
        initial_context = await self.get_context(query)
        
        # Step 2: Assess context quality
        assessment = await self.assess_context_quality(query, initial_context)
        
        # Step 3: Optimize context if needed
        if assessment["quality"] == "insufficient":
            # Try to get more context
            expanded_context = await self.get_context(query, max_results=10)
            optimized_context = await self.optimize_context(query, expanded_context)
        elif assessment["quality"] == "excessive":
            # Reduce context to most relevant
            optimized_context = await self.optimize_context(query, initial_context)
        else:
            optimized_context = initial_context
        
        # Step 4: Final assessment
        final_assessment = await self.assess_context_quality(query, optimized_context)
        
        # Step 5: Generate answer
        if final_assessment["can_answer"]:
            answer_prompt = f"""
            Query: {query}
            
            Context:
            {chr(10).join([f"- {fact}" for fact in optimized_context])}
            
            Provide a helpful answer based on the context above.
            """
            
            answer_response = await self.llm.ainvoke([HumanMessage(content=answer_prompt)])
            answer = answer_response.content
        else:
            answer = "I don't have enough relevant information to answer your question properly."
        
        return {
            "query": query,
            "initial_context_count": len(initial_context),
            "final_context_count": len(optimized_context),
            "context_quality": final_assessment["quality"],
            "confidence": final_assessment["confidence"],
            "reasoning": final_assessment["reasoning"],
            "context_facts": optimized_context,
            "answer": answer
        }

In [5]:
# Tool for external use
@tool
async def smart_search(query: str) -> str:
    """Search with intelligent context optimization"""
    agent = SimpleContextAgent()
    result = await agent.answer_with_context(query)
    
    return f"""
    Context Quality: {result['context_quality']} (confidence: {result['confidence']:.2f})
    Context Used: {result['final_context_count']} facts
    Reasoning: {result['reasoning']}
    
    Answer: {result['answer']}
    """

In [8]:
# Example usage
agent = SimpleContextAgent()

# Test with different types of queries
queries = [
    "What are the best running shoes?",
    "Tell me about customer service policies",
    "How do I return a product?"
]

for query in queries:
    print(f"\n{'='*50}")
    print(f"Query: {query}")
    print('='*50)
    
    result = await agent.answer_with_context(query)
    
    print(f"Initial Context: {result['initial_context_count']} facts")
    print(f"Final Context: {result['final_context_count']} facts")
    print(f"Quality: {result['context_quality']}")
    print(f"Confidence: {result['confidence']:.2f}")
    print(f"Reasoning: {result['reasoning']}")
    print(f"Answer: {result['answer']}")


Query: What are the best running shoes?
Initial Context: 5 facts
Final Context: 5 facts
Quality: sufficient
Confidence: 0.80
Reasoning: Reasonable amount
Answer: Based on the context provided, the best running shoes mentioned include:

1. **Men's SuperLight Wool Runners - Dark Grey (Medium Grey Sole)**: These shoes are designed for men and are made from lightweight wool, which can provide comfort and breathability during runs.

2. **TinyBirds Wool Runners - Little Kids - Natural Black (Blizzard Sole)**: These shoes are suitable for little kids and also made from wool, offering a cozy fit for young runners. They come in a size variant of 7T.

While the context does not provide a comprehensive list of all the best running shoes available, these options highlight some quality choices for both men and children. When selecting running shoes, consider factors such as fit, comfort, and the specific needs of your running style.

Query: Tell me about customer service policies
Initial Context: 

<coroutine object demo at 0x0000022959465240>